# 🚀 SelfAgent (Java版) - 在 Colab 构建 Android APK

原生 Java/Android 实现，无需 JS Bundle，构建更简单可靠！

点击 **Runtime -> Run all** 一键构建

In [ ]:
# 1. 安装 Java 17 和 Android SDK
!sudo apt-get update -qq
!sudo apt-get install -y openjdk-17-jdk wget unzip -qq

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

!java -version
print("✅ Java 17 安装完成")

In [ ]:
# 2. 安装 Android SDK
import os

!mkdir -p ~/android-sdk/cmdline-tools
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip -O cmdline-tools.zip
!unzip -q -o cmdline-tools.zip -d ~/android-sdk/cmdline-tools
!mv ~/android-sdk/cmdline-tools/cmdline-tools ~/android-sdk/cmdline-tools/latest 2>/dev/null || true

os.environ['ANDROID_HOME'] = os.path.expanduser('~/android-sdk')
os.environ['ANDROID_SDK_ROOT'] = os.environ['ANDROID_HOME']
os.environ['PATH'] = os.environ['ANDROID_HOME'] + '/cmdline-tools/latest/bin:' + os.environ['ANDROID_HOME'] + '/platform-tools:' + os.environ['PATH']

!yes | ~/android-sdk/cmdline-tools/latest/bin/sdkmanager --licenses > /dev/null 2>&1
!~/android-sdk/cmdline-tools/latest/bin/sdkmanager "platform-tools" "platforms;android-34" "build-tools;34.0.0" > /dev/null 2>&1
print("✅ Android SDK 安装完成")

In [ ]:
# 3. 克隆项目
!rm -rf python-calculator-android
!git clone https://github.com/Michael-YuQ/python-calculator-android.git
%cd python-calculator-android/selfagent_java
print("✅ 项目克隆完成")

In [ ]:
# 4. 创建 Gradle Wrapper
!mkdir -p gradle/wrapper

# 下载 gradle-wrapper.jar
!wget -q https://raw.githubusercontent.com/gradle/gradle/v8.2.0/gradle/wrapper/gradle-wrapper.jar -O gradle/wrapper/gradle-wrapper.jar

# 创建 gradlew 脚本
gradlew_content = '''#!/bin/sh
DIRNAME=$(dirname "$0")
exec java -jar "$DIRNAME/gradle/wrapper/gradle-wrapper.jar" "$@"
'''

with open('gradlew', 'w') as f:
    f.write(gradlew_content)
!chmod +x gradlew

# 使用系统 gradle 初始化 wrapper
!apt-get install -y gradle -qq
!gradle wrapper --gradle-version 8.2
print("✅ Gradle Wrapper 创建完成")

In [ ]:
# 5. 构建 Release APK
import os
os.environ['ANDROID_HOME'] = os.path.expanduser('~/android-sdk')
os.environ['ANDROID_SDK_ROOT'] = os.environ['ANDROID_HOME']
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'

!./gradlew assembleRelease --no-daemon
print("✅ APK 构建完成")

In [ ]:
# 6. 下载 APK
from google.colab import files
import shutil
import os

apk_path = 'app/build/outputs/apk/release/app-release-unsigned.apk'
if os.path.exists(apk_path):
    shutil.copy(apk_path, 'SelfAgent-Java.apk')
    print(f"APK 大小: {os.path.getsize('SelfAgent-Java.apk') / 1024 / 1024:.2f} MB")
    files.download('SelfAgent-Java.apk')
    print("✅ APK 已下载")
else:
    print("❌ APK 未找到，查找其他位置...")
    !find . -name "*.apk" -type f